<div style="
    background: linear-gradient(135deg, #4A90E2, #007AFF);
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.15);
">
  <h2 style="color:white; text-align:center; font-size:22px; margin:4px 0;">
     Projet : Optimisation des tournées de livraison avec contraintes
  </h2>
</div>

<h3 style="color:#333; font-size:18px; margin-top:16px;">Objectif du projet</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.5;
    text-align: justify;
">
L’objectif de ce projet est de modéliser et de résoudre un **problème de tournées de véhicules avec capacité (CVRP (Capacitated Vehicle Routing Problem))**.  
Nous cherchons à **minimiser la distance totale parcourue** tout en respectant les contraintes de capacité des camions et en assurant le passage unique chez chaque

Notre travail se réalise dans une démarche de **mobilité durable**, en lien avec les problématiques actuelles d’optimisation des livraisons pour réduire les coûts et les émissions de CO₂.
</div>


<div style="
    background: linear-gradient(135deg, #4A90E2, #007AFF);
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.15);
">
  <h2 style="color:white; text-align:center; font-size:22px; margin:4px 0;">
     Contexte
  </h2>
</div>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.5;
    text-align: justify;
">
  <p>
  L’ADEME a récemment lancé un appel à manifestation d’intérêt visant à promouvoir le développement de nouvelles solutions de mobilité durable, destinées aux personnes et aux marchandises, et adaptées à différents types de territoires.
  </p>
  <p>
  Dans ce cadre, les enjeux majeurs consistent à réduire les coûts tout en limitant les émissions liées aux transports. 
  Notre équipe <strong>CesiCDP</strong> a répondu à cet appel. Plus précisément, nous cherchons à concevoir un algorithme d’optimisation capable de déterminer, sur un réseau routier donné, une tournée reliant un ensemble de clients à partir d’un seul dépôt, puis revenant à ce point de départ, de manière à minimiser le temps.
  </p>
</div>


<div style="
    background: linear-gradient(135deg, #4A90E2, #007AFF);
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.15);
">
  <h2 style="color:white; text-align:center; font-size:22px; margin:4px 0;">
     Introduction
  </h2>
</div>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.5;
    text-align: justify;
">
Ce livrable présente la <strong>modélisation mathématique</strong> du problème de tournées de véhicules (VRP).  
Nous parlerons des variantes réalistes (fenêtres de temps, flotte hétérogène, trafic dépendant du temps), de la démonstration synthétique de la complexité, et des méthodes de résolution pertinentes (exactes, heuristiques, métaheuristiques).

Le but est d’expliquer de manière rigoureuse comment formaliser le problème, quelles formulations peuvent être employées en Recherche Opérationnelle, et quels outils ou solveurs sont les plus appropriés selon la taille des instances, allant de 50 à 2000 clients.
</div>


<div style="
    background: linear-gradient(135deg, #4A90E2, #007AFF);
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.15);
">
  <h2 style="color:white; text-align:center; font-size:22px; margin:4px 0;">
     Problématique
  </h2>
</div>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.5;
    text-align: justify;
">
  <blockquote style="
      border-left: 4px solid #4A90E2;
      margin: 0;
      padding-left: 12px;
      color: #333;
      font-style: italic;
  ">
  Dans le cadre de la <strong>transition écologique</strong>, comment modéliser et résoudre notre problème de tournées de livraison sur un réseau routier soumis aux contraintes imposées ?
  </blockquote>

  <blockquote style="
      border-left: 4px solid #4A90E2;
      margin-top: 10px;
      padding-left: 12px;
      color: #333;
      font-style: italic;
  ">
  Compte tenu du caractère <strong>combinatoire</strong> du problème, quelles approches issues de la <em>Recherche Opérationnelle</em> peuvent permettre d’obtenir des solutions proches de l’optimal pour des instances de grande taille (plusieurs centaines à milliers de clients) dans des temps de calcul raisonnables ?
  </blockquote>
</div>


<div style="
    background: linear-gradient(135deg, #4A90E2, #007AFF);
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.15);
">
  <h1 style="color:white; text-align:center; font-size:22px; margin:4px 0;">
     Modélisation en Recherche Opérationnelle
  </h1>
</div>

<h3 style="color:#333; font-size:18px; margin-top:16px;">1. Données et notations</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.5;
    text-align: justify;
">
Soit un graphe complet $G=(V,A)$ avec : <br>
- $V=\{0,1,\dots,n\}$ : ensemble des nœuds, où $0$ est le dépôt et  $C=V\setminus\{0\}$ l'ensemble des clients. <br>
    
- $d_{ij}\ge 0$ : distance allant du nœud $i$ au nœud $j$.
  - Les coordonnées sont $(x_i,y_i)$ et type `Euclidien_2D` :
    $$ d_{ij} = \big(\sqrt{(x_i-x_j)^2+(y_i-y_j)^2}\big) $$
- $q_i\ge 0$ : demande du client $i$ (avec $d_0=0$).
- $Q$ : capacité du véhicule.
- $K$ : nombre de véhicules disponibles (optionnel).
-  $[a_i,b_i]$ : fenêtres temporelles 
- $s_i$ : temps de service
- $c_{ij}(t)$ : fonction de trafic
</div>

<div style="
    background: linear-gradient(135deg, #4A90E2, #007AFF);
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.15);
">
  <h2 style="color:white; text-align:center; font-size:22px; margin:4px 0;">
     2. Formulation de base : CVRP (formulation flux / arêtes)
  </h2>
</div>

<h4 style="color:#333; font-size:16px; margin-top:12px;">Variables :</h4>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 6px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
    gap: 5px
">

$$
x_{ij} =
\begin{cases}
1 & \text{si le véhicule va directement du client } i \text{ vers } j \\
0 & \text{sinon}
\end{cases}
$$

$$
y_i = \text{charge totale livrée avant d’arriver au client } i
$$

$$
d_{ij} = \text{distance (ou coût) entre les clients } i \text{ et } j
$$

$$
q_i = \text{demande du client } i
$$

$$
Q = \text{capacité maximale d’un véhicule}
$$

$$
n = \text{nombre total de clients (hors dépôt)}
$$
</div>


<h4 style="color:#333; font-size:16px; margin-top:16px;">Fonction économique</h4>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 6px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">

$$
\text{Minimiser } Z = \sum_{i=0}^{n} \sum_{j=0}^{n} d_{ij} \cdot x_{ij}
$$

L’objectif est de minimiser la distance totale parcourue par l’ensemble des véhicules.
</div>


<h4 style="color:#333; font-size:16px; margin-top:16px;">Contraintes</h4>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 6px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">

1. <strong>Chaque client est visité une seule fois :</strong>  
$$
\sum_{j=0}^{n} x_{ij} = 1 \quad \forall i = 1, \ldots, n
$$

2. <strong>Chaque client a exactement un véhicule qui arrive :</strong>  
$$
\sum_{i=0}^{n} x_{ij} = 1 \quad \forall j = 1, \ldots, n
$$

3. <strong>Flux de continuité (chaque véhicule part et revient au dépôt) :</strong>  
$$
\sum_{j=1}^{n} x_{0j} = k \quad \text{et} \quad \sum_{i=1}^{n} x_{i0} = k
$$

4. <strong>Capacité maximale du véhicule :</strong>  
$$
y_i + q_j \leq Q + (Q - q_i)(1 - x_{ij}) \quad \forall i, j
$$

5. <strong>Sous-tournées interdites (MTZ) :</strong>  
$$
y_i - y_j + Q \cdot x_{ij} \leq Q - q_j \quad \forall i \neq j
$$

6. <strong>Nature des variables :</strong>  
$$
x_{ij} \in \{0,1\}, \quad y_i \geq 0
$$
</div>


<h4 style="color:#333; font-size:16px; margin-top:16px;">Résolution graphique (illustration simplifiée)</h4>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 6px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">

Pour un petit nombre de clients (3 à 5), le problème peut être représenté graphiquement :  
<ul>
    <li>Les sommets représentent les clients</li>
    <li>Les arêtes représentent les trajets possibles</li>
    <li>Le coût est la distance</li>
</ul>

La solution optimale correspond au plus court cycle qui visite tous les clients sans dépasser la capacité \( Q \).
</div>



<div id="graph" style="background-color:#F9FAFB;
    border-left: 6px solid #4A90E2;
    padding: 16px 20px;
    border-radius: 8px;
    font-size: 17px;
    line-height: 1.6;text-align:center; margin-top:15px; margin-bottom:25px;">
  <img src="graph.jpg" alt="Graphe" width="800" style="border-radius:10px; box-shadow:0 4px 10px rgba(0,0,0,0.15);">
</div>


<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
<ul>
    <li>Meilleur tour trouvé (avec retour au dépôt) :
0 -> 1 -> 4 -> 3 -> 2 -> 5 -> 0</li>
<li>Coût total : 49</li>
</ul>
</div>


<div style="
    background: linear-gradient(135deg, #4A90E2, #007AFF);
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.15);
">
  <h2 style="color:white; text-align:center; font-size:22px; margin:4px 0;">
     4. Complexité
  </h2>
</div>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
<ul>
    <li>Cas $K=1$ ⇒ TSP métrique ⇒ NP-complet.</li>
    <li>Donc CVRP est NP-difficile.</li>
    <li>→ Nécessité d’heuristiques.</li>
</ul>
</div>


<div style="
    background: linear-gradient(135deg, #4A90E2, #007AFF);
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.15);
">
  <h2 style="color:white; text-align:center; font-size:22px; margin:4px 0;">
     Métaheuristique choisie : Algorithme Tabou Multi-Start
  </h2>
</div>

<h3 style="color:#333; font-size:18px; margin-top:16px;">Principe général</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
<strong>L’algorithme tabou </strong> est une méthode de recherche locale qui explore des solutions voisines tout en évitant de revisiter des solutions déjà explorées grâce à une **mémoire taboue**.

La version <strong>Multi-Start</strong> consiste à relancer plusieurs fois l’algorithme tabou à partir de solutions initiales différentes afin d’échapper aux minima locaux.
</div>


<h3 style="color:#333; font-size:18px; margin-top:16px;">Étapes de l’algorithme</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top:4px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
<ol>
    <li><strong>Génération de plusieurs solutions initiales</strong> (aléatoires ou construites par heuristique simple).</li>
    <li><strong>Application de la recherche taboue</strong> sur chaque solution :
        <ul>
            <li>Définir le voisinage (échanges, insertions…)</li>
            <li>Explorer les solutions voisines</li>
            <li>Interdire temporairement certains mouvements (liste taboue)</li>
            <li>Accepter les meilleures solutions selon le critère d’aspiration</li>
        </ul>
    </li>
    <li><strong>Comparaison des résultats obtenus</strong> pour chaque point de départ</li>
    <li><strong>Sélection de la meilleure solution globale</strong></li>
</ol>
</div>


<div style="
    background: linear-gradient(135deg, #4A90E2, #007AFF);
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.15);
">
  <h2 style="color:white; text-align:center; font-size:22px; margin:4px 0;">
     Points forts et faibles
  </h2>
</div>

<h3 style="color:#333; font-size:18px; margin-top:16px;">Avantages</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
<ul>
    <li>Bonne robustesse sur les instances de grande taille</li>
    <li>Facilité de paramétrage</li>
    <li>Bonne convergence vers des solutions quasi-optimales</li>
    <li>Facilement parallélisable (multi-start)</li>
</ul>
</div>

<h3 style="color:#333; font-size:18px; margin-top:16px;">Inconvénients</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
<ul>
    <li>Temps de calcul plus long si le voisinage est grand</li>
    <li>Nécessite un réglage fin de la taille de la mémoire taboue</li>
    <li>Peut dépendre de la qualité des solutions initiales</li>
</ul>
</div>


<h3 style="color:#333; font-size:18px; margin-top:16px;">Exemple de voisinage utilisé</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
<ul>
    <li><strong>Swap</strong> : échanger deux clients dans une même tournée.</li>
    <li><strong>Relocate</strong> : déplacer un client d’une tournée à une autre.</li>
    <li><strong>2-Opt</strong> : inverser l’ordre d’un sous-ensemble de clients dans une tournée.</li>
</ul>
</div>

<h3 style="color:#333; font-size:18px; margin-top:16px;">Critères d’arrêt</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
<ul>
    <li>Nombre maximal d’itérations sans amélioration</li>
    <li>Temps limite</li>
    <li>Convergence (stabilité de la meilleure solution)</li>
</ul>
</div>

<h3 style="color:#333; font-size:18px; margin-top:16px;">Résultats attendus</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
Pour un ensemble de 50 à 2000 clients, cette méthode permet d’obtenir :  
<ul>
    <li>Des écarts inférieurs à <strong>5–10 %</strong> par rapport à l’optimal connu.</li>
    <li>Des temps de calcul raisonnables (&lt; 5 minutes pour 500 clients avec une bonne implémentation).</li>
</ul>
</div>


<h3 style="color:#333; font-size:18px; margin-top:16px;">7. Implémentation</h3>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
<ul>
    <li><code>numpy</code> pour la matrice de distances</li>
    <li>Éviter copies profondes</li>
    <li>Paralléliser runs (multi-cœurs)</li>
</ul>
</div>


<h3 style="color:#333; font-size:18px; margin-top:16px;">8. Références</h3>


<div style="background-color:#4A90E2; padding:12px; border-radius:10px; margin-bottom:15px;">
  <h2 style="color:white; text-align:center; margin:0;"> Bibliographie</h2>
</div>
<div style="background-color:#EAF2FB; border-left:6px solid #4A90E2; padding:20px; border-radius:12px; box-shadow:0px 4px 8px rgba(0,0,0,0.1); font-size:17px; line-height:1.8;">
  <ol style="padding-left:25px; color:#2C3E50;">
    <li id="biblio1">
      <b>Benjamin Hellouin</b>, <i>Problème du voyageur de commerce : cas métrique</i>, 
      document pédagogique, 
      <a href="https://github.com/PyVRP/VRPLIB" target="_blank"><b>Approx-TSP-métrique.pdf</b></a>, 2022.
    </li>
      <b>Leon Lan</b>, <i>VRPLIB – Python package to read and write vehicle routing problem instances</i>, 
      <a href="https://github.com/PyVRP/VRPLIB" target="_blank"><b>GitHub - PyVRP/VRPLIB.</b></a>
    </li>
      <li id="biblio3"><b>Maryline Lakh</b>  <i> Problème de tournées de véhicules : défis, solutions et exemples concrets </i> : <a href="https://antsroute.com/blog/probleme-de-tournees-de-vehicules-defis-solutions" target="_blank"><b>Problèmes de tournées de véhicules : définition, défis et solutions</b></a></li>
  </ol>
</div>


<div style="
    background: linear-gradient(135deg, #4A90E2, #007AFF);
    padding: 10px;
    border-radius: 10px;
    box-shadow: 0 2px 6px rgba(0,0,0,0.15);
">
  <h2 style="color:white; text-align:center; font-size:22px; margin:4px 0;">
     Conclusion
  </h2>
</div>

<div style="
    background-color:#F9FAFB;
    border-left: 5px solid #4A90E2;
    padding: 12px 18px;
    border-radius: 6px;
    margin-top: 8px;
    font-size: 15px;
    line-height: 1.6;
    text-align: justify;
">
<p>
La combinaison de la <strong>modélisation en recherche opérationnelle</strong> et de la <strong>métaheuristique Tabou Multi-Start</strong> permet de traiter efficacement le problème du <strong>CVRP (Capacitated Vehicle Routing Problem)</strong>.
</p>

<ul>
    <li>La modélisation formelle garantit une compréhension rigoureuse du problème et de ses contraintes.</li>
    <li>La méthode tabou, enrichie par le multi-start, fournit une solution proche de l’optimal, même sur de grandes instances.</li>
</ul>

<p>
Ce projet démontre la pertinence d’une approche hybride mêlant théorie et heuristique pour répondre à des problématiques réelles de logistique et de transport durable.
</p>
</div>
